In [1]:
import numpy as np
import matplotlib.pyplot as plt
from utils import load_imagenet64, one_hot, make4D
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm, trange
from sklearn.metrics import accuracy_score
import time
import torchvision
from resnet import ResNet152

GPU = torch.device('cuda:0')
CPU = torch.device('cpu')

In [6]:
import os

os.listdir("/mnt/data/datasets/ILSVRC/train_npy")

OSError: [Errno 5] Input/output error: '/mnt/data/datasets/ILSVRC/train_npy'

In [ ]:
import json
with open("labels.json") as f:
	labels = json.load(f)

In [ ]:
(x, y), (x_val, y_val) = load_imagenet(2)  # 5 parts of 10 parts
m = len(x)

In [ ]:
y[0].numpy().argmax()
# y[0].argmax(1)

In [ ]:
# a[0].shape

In [ ]:
# Show a sample image
a = x.numpy().transpose(0, 2, 3, 1)/255
print(labels[f"{y[0].numpy().argmax()}"])
plt.imshow(a[0])

In [ ]:
x.shape, y.shape, x_val.shape, y_val.shape

In [ ]:
hparams = {
    "batch_size": 256,
    "num_epochs": 40,
    "model": "resnet",
    "dataset": "imagenet",
    "optimizer": "sgd",
    "learning_rate": 0.1,  # 1.2e-5, 1.2e-7 can be tried
}

# model = torchvision.models.resnet152()
model = ResNet152()

model.to(GPU)
optimiser = torch.optim.SGD(model.parameters(), lr=hparams["learning_rate"])
loss_fn = nn.CrossEntropyLoss()
losses = []

In [ ]:
batch_size = hparams["batch_size"]
num_epochs = hparams["num_epochs"]

ttra, tacc = 0, 0

for epoch in range(num_epochs):

    pbar = tqdm(total=m//batch_size+1, desc=f"Epoch {epoch+1}/{num_epochs}")
    
    t0 = time.time()

    for i in range(0, len(x), batch_size):
        batch_x, batch_y = x[i:i+batch_size].to(GPU), y[i:i+batch_size].to(GPU)
        optimiser.zero_grad()
        output = model(batch_x)
        loss = loss_fn(output, batch_y.argmax(1))
        loss.backward()
        optimiser.step()
        pbar.update(1)

    t1 = time.time()

    train_accuracy = accuracy_score(y[:5000].argmax(1),     model(x[:5000].to(GPU)).to(CPU).argmax(1))
    test_accuracy  = accuracy_score(y_val[:5000].argmax(1), model(x_val[:5000].to(GPU)).to(CPU).argmax(1))

    t2 = time.time()

    ttra += t1-t0
    tacc += t2-t1

    losses.append(loss.item())
    torch.save(model, "resnet_imagenet.pth")
    pbar.close()

In [ ]:
ttra, tacc

In [ ]:
plt.plot(np.arange(len(losses)), losses)

In [ ]:
# torch.save(model, "resnet_imagenet.pth")

In [ ]:
model = torch.load("resnet_imagenet.pth")

In [ ]:
model = model.to(CPU)

In [ ]:
y_pred = model(x[:1000])
y_pred_val = model(x_val[:1000])

train_accuracy = accuracy_score(y[:1000].argmax(1), y_pred.argmax(1))
test_accuracy = accuracy_score(y_val[:1000].argmax(1), y_pred_val.argmax(1))

In [ ]:
print(f"Train accuracy: {train_accuracy*100}")
print(f"Test accuracy: {test_accuracy*100}")

In [ ]:
from utils import load_imagenet
import torch
from tqdm import trange
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import time

hparams = {
    "batch_size": 5000,
    "num_epochs": 40,
    "model": "resnet152 from torchvision.models",
    "dataset": "imagenet",
    "optimizer": "sgd",
    "learning_rate": 0.1,
    "gpu": 0,
}

GPU = torch.device(f"cuda:{hparams['gpu']}")
CPU = torch.device('cpu')

model = torch.load("/home/aritra/project/quartLT23/3_imagenet_resnet/B=256_E=40_O=sgd.pth").to(GPU)


print("Loading data...")
(x, y), (x_val, y_val) = load_imagenet(1)
m = len(x)

times = []
batch_sizes = []
for batch_size in trange(550, 700, 5):
    try:
        n = m // batch_size + 1
        t0 = time.time()
        for i in range(5):
            batch_x = x[:batch_size].to(GPU)
            batch_y = y[:batch_size]
            accuracy_score(batch_y.argmax(1), model(batch_x.to(GPU)).to(CPU).argmax(1))
        times.append((time.time() - t0) * n/3)
        batch_sizes.append(batch_size)
    except:
        break


plt.plot(batch_sizes, times)
plt.xlabel("Batch size")
plt.ylabel("Time (s)")

In [ ]:
import numpy as np
from utils import load_imagenet
import torch
from tqdm import trange
from sklearn.metrics import accuracy_score

hparams = {
    "batch_size": 630,
    "num_epochs": 40,
    "model": "resnet152 from torchvision.models",
    "dataset": "imagenet",
    "optimizer": "sgd",
    "learning_rate": 0.1,
    "gpu": 1,
}

GPU = torch.device(f"cuda:{hparams['gpu']}")
CPU = torch.device('cpu')

model = torch.load("/home/aritra/project/quartLT23/3_imagenet_resnet/B=256_E=40_O=sgd.pth").to(GPU)


print("Loading data...")
(x, y), (x_val, y_val) = load_imagenet()
m = len(x)

batch_size = hparams["batch_size"]

train_accuracy, test_accuracy = [], []

In [ ]:
for i in trange(0, len(x), batch_size):
    batch_x, batch_y = x[i:i+batch_size].to(GPU), y[i:i+batch_size]
    train_accuracy.append(accuracy_score(batch_y.argmax(1), model(batch_x).to(CPU).argmax(1)))
print(f"Train accuracy: {np.mean(train_accuracy)*100}%")

In [ ]:
for i in trange(0, len(x), batch_size):
    batch_x_val, batch_y_val = x_val[i:i+batch_size].to(GPU), y_val[i:i+batch_size]
    test_accuracy.append(accuracy_score(batch_y_val.argmax(1), model(batch_x_val).to(CPU).argmax(1)))
print(f"Test accuracy: {np.mean(test_accuracy)*100}%")

In [ ]:
(x, y), (x_val, y_val) = load_imagenet(1)

In [ ]:
x = x[:5000]

In [ ]:
plt.imshow(x[2].permute(1, 2, 0))

In [ ]:
x_ = make4D(x)

In [ ]:
x[0][:-1].shape

In [ ]:
plt.imshow(x_[2][:-1].permute(1, 2, 0), cmap="gray")